In [ ]:
!pip install pandas openpyxl

In [ ]:
import os
import pandas as pd
import re

In [ ]:
os.chdir('/home/jovyan/work')

Load the CCG list. This is the CCG21CD, and is the canonical reference for dataset creation

In [ ]:
ccg = pd.read_csv('data/ccg_lookup.csv')

Load the CCG to LAD lookup.

In [ ]:
ccg_lad_lookup = pd.read_csv('data/ccg_lad_lookup.csv')

Load the population (manually prepared file).

In [ ]:
population = pd.read_csv('data/ons-population-estimates.csv').drop(columns=[
    'CCG Name', 'STP21 Code', 'STP21 Name', 'NHSER21 Code', 'NHSER21 Name', 
])

Load the origin file and summarise by CCG. Sadly there are LAD20 LADs in the origin file, and LAD21 LADs in the reference data, so we lose some data.

In [ ]:
origin = pd.read_excel(
    './source/Origins By Local Authority 2021.xlsx',
    na_filter=False
)

origin = origin[origin.LAD20CD.str.match('^E')].drop(
    columns = [
        'NWB%',
       'AAA : ENGLISH (Weighted)', 'BAA : SCOTTISH (Weighted)',
       'BAB : WELSH (Weighted)', 'BBA : NORTHERN IRISH', 'BBB : IRISH',
       'CAA : SPANISH', 'CBA : FILIPINO', 'CCA : PORTUGESE OR BRAZILIAN',
       'DAA : FRENCH OR WALLOON', 'DCA : DUTCH OR FLEMISH', 'DDA : GERMAN',
       'DHA : ITALIAN OR MALTESE', 'DZZ : SCANDINAVIAN', 'EIA : POLISH',
       'EJZ : CZECH OR SLOVAK', 'EKA : HUNGARIAN', 'ELZ : BALTIC STATES',
       'EMF : ALBANIAN', 'EMZ : FORMERLY YUGOSLAV', 'ENA : BULGARIAN',
       'EOA : ROMANIAN', 'EPZ : RUSSIAN OR UKRAINIAN',
       'FAZ : GREEK OR GREEK CYPRIOT', 'GAA : JEWISH', 'GBA : ARMENIAN',
       'HAA : BLACK CARIBBEAN (Weighted)', 'HBA : NIGERIAN', 'HCA : GHANAIAN',
       'HDZ : BLACK SOUTH AFRICAN', 'HED : ETHIOPIAN',
       'HZZ : OTHER BLACK AFRICAN', 'IAZ : NORTH AFRICAN MUSLIM',
       'IBZ : SOMALI', 'ICA: TURKISH', 'IFA : IRANIAN', 'IHZ : PAKISTANI',
       'IIZ : KASHMIRI', 'IKA : BANGLADESHI', 'IZZ : OTHER MUSLIM',
       'JAA : SIKH', 'KAA : HINDU INDIAN', 'KBA : TAMIL OR SRI LANKAN',
       'KCA : BANGLADESHI HINDU', 'LAA : JAPANESE', 'LAB : KOREAN',
       'LBA : MANDARIN CHINESE', 'LBB : CANTONESE CHINESE', 'LCA : VIETNAMESE',
       'LZZ : OTHER EAST ASIAN', 'UNKNOWN', 'XXX : UNCLASSIFIED AND OTHER',
       'YYY : NOT RECOGNISED'
    ]
)
origin = origin.merge(ccg_lad_lookup, how='right', left_on=['LAD20CD'], right_on=['LAD21CD'])
origin = origin.groupby('CCG21CD').agg(sum)
origin['NWB_pct'] = pd.Series(origin['NWB'] / origin['Population Total'] * 100).round(2)
origin.to_csv('data/origin.csv')

Load the trust lookup file - this was derived from the IAPT file.

In [ ]:
trust_lookup = pd.read_csv('data/trust_ccg_lookup.csv').drop(columns=['trust_name', 'ccg_name'])

Read the CMHS data and discard some columns, then summarise by CCG.

In [ ]:
cmhs = pd.read_excel(
    'source/CMHS.xlsx'
).merge(
    trust_lookup, left_on='Trust Code', right_on='trust_code'
).merge(
    ccg, on='CCG21CDH'
).drop(
    columns=['trust_code', 'Trust Code', 'Trust Name', 'CCG21CDH', 'CCG21NM']
)

cmhs = cmhs.drop(
    columns=[x for x in cmhs.columns if not re.match("^(meanQ|CCG)", x)]
).groupby('CCG21CD').agg('mean')

Load the MHSDS data

In [ ]:
mhsds = pd.read_csv('source/MHSDS Data_Rstr_Sep_Nov_2020.csv', na_filter=False, header=[0,1])
mhsds.columns = mhsds.columns.map('_'.join)
mhsds.columns = [
    'trust_code',
    'reporting_period_start',
    'Chemical restraint - Injection (Non Rapid Tranquillisation)_BAME',
    'Chemical restraint - Injection (Non Rapid Tranquillisation)_Non-BAME',
    'Chemical restraint - Injection (Non Rapid Tranquillisation)_Unknown/Not Stated',
    'Chemical restraint - Injection (Rapid Tranquillisation)_BAME',
    'Chemical restraint - Injection (Rapid Tranquillisation)_Non-BAME',
    'Chemical restraint - Injection (Rapid Tranquillisation)_Unknown/Not Stated',
    'Chemical restraint - Oral_BAME', 'Chemical restraint - Oral_Non-BAME',
    'Chemical restraint - Oral_Unknown/Not Stated',
    'Chemical restraint - Other (not listed)_BAME',
    'Chemical restraint - Other (not listed)_Non-BAME',
    'Chemical restraint - Other (not listed)_Unknown/Not Stated',
    'Chemical Restraint_BAME', 'Chemical Restraint_Non-BAME',
    'Chemical Restraint_Unknown/Not Stated', 'Mechanical restraint_BAME',
    'Mechanical restraint_Non-BAME',
    'Mechanical restraint_Unknown/Not Stated',
    'Physical restraint - Kneeling_BAME',
    'Physical restraint - Kneeling_Non-BAME',
    'Physical restraint - Kneeling_Unknown/Not Stated',
    'Physical restraint - Other (not listed)_BAME',
    'Physical restraint - Other (not listed)_Non-BAME',
    'Physical restraint - Other (not listed)_Unknown/Not Stated',
    'Physical restraint - Prone_BAME',
    'Physical restraint - Prone_Non-BAME',
    'Physical restraint - Prone_Unknown/Not Stated',
    'Physical restraint - Restrictive escort_BAME',
    'Physical restraint - Restrictive escort_Non-BAME',
    'Physical restraint - Restrictive escort_Unknown/Not Stated',
    'Physical restraint - Seated_BAME',
    'Physical restraint - Seated_Non-BAME',
    'Physical restraint - Seated_Unknown/Not Stated',
    'Physical restraint - Side_BAME', 'Physical restraint - Side_Non-BAME',
    'Physical restraint - Side_Unknown/Not Stated',
    'Physical restraint - Standing_BAME',
    'Physical restraint - Standing_Non-BAME',
    'Physical restraint - Standing_Unknown/Not Stated',
    'Physical restraint - Supine_BAME',
    'Physical restraint - Supine_Non-BAME',
    'Physical restraint - Supine_Unknown/Not Stated',
    'Physical Restraint_BAME', 'Physical Restraint_Non-BAME',
    'Physical Restraint_Unknown/Not Stated', 'Seclusion_BAME',
    'Seclusion_Non-BAME', 'Seclusion_Unknown/Not Stated',
    'Segregation_BAME', 'Segregation_Non-BAME',
    'Segregation_Unknown/Not Stated', 'UNKNOWN_BAME', 'UNKNOWN_Non-BAME',
    'UNKNOWN_Unknown/Not Stated']
mhsds = mhsds[~mhsds.trust_code.str.match('.*Average')].drop(columns=[
       'Chemical restraint - Injection (Non Rapid Tranquillisation)_BAME',
       'Chemical restraint - Injection (Non Rapid Tranquillisation)_Non-BAME',
       'Chemical restraint - Injection (Non Rapid Tranquillisation)_Unknown/Not Stated',
       'Chemical restraint - Injection (Rapid Tranquillisation)_BAME',
       'Chemical restraint - Injection (Rapid Tranquillisation)_Non-BAME',
       'Chemical restraint - Injection (Rapid Tranquillisation)_Unknown/Not Stated',
       'Chemical restraint - Oral_BAME', 'Chemical restraint - Oral_Non-BAME',
       'Chemical restraint - Oral_Unknown/Not Stated',
       'Chemical restraint - Other (not listed)_BAME',
       'Chemical restraint - Other (not listed)_Non-BAME',
       'Chemical restraint - Other (not listed)_Unknown/Not Stated',
       'Physical restraint - Kneeling_BAME',
       'Physical restraint - Kneeling_Non-BAME',
       'Physical restraint - Kneeling_Unknown/Not Stated',
       'Physical restraint - Other (not listed)_BAME',
       'Physical restraint - Other (not listed)_Non-BAME',
       'Physical restraint - Other (not listed)_Unknown/Not Stated',
       'Physical restraint - Prone_BAME',
       'Physical restraint - Prone_Non-BAME',
       'Physical restraint - Prone_Unknown/Not Stated',
       'Physical restraint - Restrictive escort_BAME',
       'Physical restraint - Restrictive escort_Non-BAME',
       'Physical restraint - Restrictive escort_Unknown/Not Stated',
       'Physical restraint - Seated_BAME',
       'Physical restraint - Seated_Non-BAME',
       'Physical restraint - Seated_Unknown/Not Stated',
       'Physical restraint - Side_BAME', 'Physical restraint - Side_Non-BAME',
       'Physical restraint - Side_Unknown/Not Stated',
       'Physical restraint - Standing_BAME',
       'Physical restraint - Standing_Non-BAME',
       'Physical restraint - Standing_Unknown/Not Stated',
       'Physical restraint - Supine_BAME',
       'Physical restraint - Supine_Non-BAME',
       'Physical restraint - Supine_Unknown/Not Stated'
])

In [ ]:
mhsds = mhsds.merge(trust_lookup, how='left', on='trust_code').merge(ccg, how='left', on='CCG21CDH').groupby('CCG21CD').agg('mean')

In [ ]:
data = ccg.merge(
    population,
    how='left',
    left_on='CCG21CD',
    right_on='CCG Code',
).drop(
    columns=['CCG Code']
)
data = data.merge(
    origin,
    how='left',
    on='CCG21CD'
)
data = data.merge(
    cmhs,
    how='left',
    on='CCG21CD',
)
data = data.merge(mhsds,
    how='left',
    on='CCG21CD'
)

Save the data as a json file

In [ ]:
data.to_json('data/data.json')